# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [1]:
# Use the following data for this assignment:

import pandas as pd
import numpy as np

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])
df

,0,1,2,3,4,5,6,7,8,9,...,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649
1992,-8941.531897,127788.667612,-71887.743011,-79146.060869,425156.114501,310681.166595,50581.575349,88349.230566,185804.513522,281286.947277,...,171938.760289,150650.759924,203663.976475,-377877.158072,-197214.093861,24185.008589,-56826.729535,-67319.766489,113377.299342,-4494.878538
1993,-51896.094813,198350.518755,-123518.252821,-129916.759685,216119.147314,49845.883728,149135.648505,62807.672113,23365.577348,-109686.264981,...,-44566.520071,101032.122475,117648.199945,160475.622607,-13759.888342,-37333.493572,103019.841174,179746.127403,13455.493990,34442.898855
1994,152336.932066,192947.128056,389950.263156,-93006.152024,100818.575896,5529.230706,-32989.370488,223942.967178,-66721.580898,47826.269111,...,165085.806360,74735.174090,107329.726875,199250.734156,-36792.202754,-71861.846997,26375.113219,-29328.078384,65858.761714,-91542.001049
1995,-69708.439062,-13289.977022,-30178.390991,55052.181256,152883.621657,12930.835194,63700.461932,64148.489835,-29316.268556,59645.677367,...,-13901.388118,50173.686673,53965.990717,4128.990173,72202.595138,39937.199964,139472.114293,59386.186379,73362.229590,28705.082908


In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation
from matplotlib.widgets import Slider
#from matplotlib.widgets import TextBox
import math
import matplotlib.patches as patches
from ipywidgets import widgets, interactive
%matplotlib notebook

df['Mean']=df.apply(np.mean,axis=1)
df['SE of Mean']=df.sem(axis=1)
df['Error']=df['SE of Mean']*1.96

df

,0,1,2,3,4,5,6,7,8,9,...,3643,3644,3645,3646,3647,3648,3649,Mean,SE of Mean,Error
1992,-8941.531897,127788.667612,-71887.743011,-79146.060869,425156.114501,310681.166595,50581.575349,88349.230566,185804.513522,281286.947277,...,-377877.158072,-197214.093861,24185.008589,-56826.729535,-67319.766489,113377.299342,-4494.878538,33312.107476,3319.956609,6507.114954
1993,-51896.094813,198350.518755,-123518.252821,-129916.759685,216119.147314,49845.883728,149135.648505,62807.672113,23365.577348,-109686.264981,...,160475.622607,-13759.888342,-37333.493572,103019.841174,179746.127403,13455.493990,34442.898855,41861.859541,1628.255022,3191.379843
1994,152336.932066,192947.128056,389950.263156,-93006.152024,100818.575896,5529.230706,-32989.370488,223942.967178,-66721.580898,47826.269111,...,199250.734156,-36792.202754,-71861.846997,26375.113219,-29328.078384,65858.761714,-91542.001049,39493.304941,2322.783068,4552.654813
1995,-69708.439062,-13289.977022,-30178.390991,55052.181256,152883.621657,12930.835194,63700.461932,64148.489835,-29316.268556,59645.677367,...,4128.990173,72202.595138,39937.199964,139472.114293,59386.186379,73362.229590,28705.082908,47743.550969,1154.709998,2263.231596


In [3]:
%matplotlib notebook

y_target = widgets.BoundedFloatText(
    value=31560,
    min=0,
    max=60000,
    step=2500,
    description='Initial y-target',
    disabled=False,
    color='black'
)

input_range = widgets.BoundedFloatText(
    value=5000,
    min=0,
    max=20000,
    step=2500,
    description='Range',
    disabled=False,
    color='black'
)


def plotit(y_target, input_range):

    def round_up(value):
        value_tenthousands=value/10000
        return math.ceil(value_tenthousands)*10000

    fig=plt.figure(figsize=(7,7))
    ax=fig.add_subplot(111)
    y_limit=y_target
    range_user_input=input_range
    y_upper=y_limit+(range_user_input/2)
    y_lower=y_limit-(range_user_input/2)
    yaxis_lim=60000
    
    cmap = plt.cm.get_cmap('RdBu')
    bar=ax.bar(df.index.tolist(),df['Mean'],width=1,capsize=10,edgecolor='black',linewidth=1)
    ax.errorbar(df.index.tolist(),df['Mean'], yerr=df['Error'].tolist(), color='black', capsize=10, ls='none')
    ax.set_xticks(df.index.tolist())
    ax.set_ylim(0,yaxis_lim)
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    ax.set_title('Interactive Chart\nY-value Vs Year', fontweight='bold', fontsize='16')
    ax.set_ylabel('y-value', fontweight='bold',fontsize=12)
    ax.set_xlabel('Year', fontweight='bold',fontsize=12)

    def cmap_calculate(yvalue_lower,yvalue_upper,y_lower,y_upper):
        #Calculates cmap_value
        yvalue_range=np.arange(yvalue_lower,yvalue_upper+1).astype('int')
        y_range=np.arange(y_lower,y_upper+1).astype('int')
        yvalue_range.sort()
        y_range.sort()
        if any(np.in1d(yvalue_range,y_range)):
            cmap_value=int(cmap.N/2)
            return cmap_value
        elif y_range.max()<yvalue_range.min():
            cmap_value=int(cmap.N/2+((y_range.max()-yvalue_range.min())/(yaxis_lim/5))*(cmap.N/2))
            return cmap_value
        elif y_range.min()>yvalue_range.max():
            cmap_value=int(cmap.N/2+((y_range.min()-yvalue_range.max())/(yaxis_lim/5))*(cmap.N/2))
            return cmap_value


    for i in range(0,len(bar)):
       # cmap_value=int(cmap.N/2+(df['Mean'].iloc[i]-y_limit)*(cmap.N/2)/(y_limit/4))
        yvalue_lower=df['Mean'].iloc[i]-df['Error'].iloc[i]
        yvalue_upper=df['Mean'].iloc[i]+df['Error'].iloc[i]
        cmap_value=cmap_calculate(yvalue_lower,yvalue_upper,y_lower,y_upper)
        bar[i].set_color(cmap(cmap_value))
        bar[i].set_linewidth(1)
        bar[i].set_edgecolor('black')


    #Slider Axis Mod  
    slider_ax_ymin=ax.get_position().y0+((range_user_input/2)/yaxis_lim)*(ax.get_position().y1-ax.get_position().y0)
    slider_ax_yheight=((yaxis_lim-range_user_input)/yaxis_lim)*(ax.get_position().y1 - ax.get_position().y0)
    slider_ax=plt.axes([ax.get_position().x0,slider_ax_ymin, ax.get_position().width,slider_ax_yheight])
    slider_ax.spines['top'].set_visible(False)
    slider_ax.spines['bottom'].set_visible(False)

    slider_ax.patch.set_alpha(0)
    hline=ax.axhline(y=y_limit,color='r',alpha=0.5,linestyle='--')
    hline_upper=ax.axhline(y=y_upper,color='r',alpha=0.7)
    hline_lower=ax.axhline(y=y_lower,color='r',alpha=0.7)
    hline_shaded_range=df.index.tolist()
    hline_shaded_range.sort()
    hline_shaded_range=np.array(hline_shaded_range)
    rect = patches.Rectangle((hline_shaded_range.min()-1,y_lower),
                    hline_shaded_range.max()-hline_shaded_range.min()+2,y_upper-y_lower,linewidth=1,facecolor='lightgray',alpha=0.5)
    ax.add_patch(rect)

    a_slider = Slider(ax=slider_ax,      
                      label='',           
                      valmin=range_user_input/2,          
                      valmax=60000-range_user_input/2,          
                      valinit=y_limit,  
                      valstep=10,
                      orientation='vertical',
                      color='black',
                      alpha=0,
                      edgecolor='black',
                      valfmt='%0.0f'
                     )
    #a_slider.valtext.set_text('{:}'.format(int(a_slider.valtext.get_text())))
    a_slider.hline.set_alpha(0)
    #a_slider.valtext.set_position((1.06,(a_slider.val/yaxis_lim+0.02)))
    a_slider.valtext.set_alpha(0)
    upper_text=slider_ax.text(1.02,y_upper,'{:}'.format(int(y_upper)))
    lower_text=slider_ax.text(1.02,y_lower,'{:}'.format(int(y_lower)))
    slider_text=slider_ax.text(1.02,y_limit,'{:}'.format(int(y_limit)))


    def update(val):
        for i in range(0,len(bar)):
            yvalue_lower=df['Mean'].iloc[i]-df['Error'].iloc[i]
            yvalue_upper=df['Mean'].iloc[i]+df['Error'].iloc[i]
            y_upper=a_slider.val+(range_user_input/2)
            y_lower=a_slider.val-(range_user_input/2)
            cmap_value=cmap_calculate(yvalue_lower,yvalue_upper,y_lower,y_upper)
            bar[i].set_color(cmap(cmap_value))
            bar[i].set_linewidth(1)
            bar[i].set_edgecolor('black')
            hline.set_ydata(a_slider.val)
            hline_upper.set_ydata(y_upper)
            hline_lower.set_ydata(y_lower)
            ax.patches[4].set_y(y_lower)
            #a_slider.valtext.set_position((1.06,(a_slider.val/yaxis_lim+0.02)))
            upper_text.set_text(int(y_upper))
            lower_text.set_text(int(y_lower))
            slider_text.set_text(int(a_slider.val))
            upper_text.set_position((1.02,y_upper))
            lower_text.set_position((1.02,y_lower))
            slider_text.set_position((1.02,a_slider.val))
    a_slider.on_changed(update)
    
interactive(plotit,y_target=y_target, input_range=input_range)

interactive(children=(BoundedFloatText(value=31560.0, description='Initial y-target', max=60000.0, step=2500.0…